# Today's Lesson? Data Science is mostly Data Cleaning

![data_cleaning_1](data_cleaning_1.png)
[https://www.nytimes.com/2014/08/18/technology/for-big-data-scientists-hurdle-to-insights-is-janitor-work.html](https://www.nytimes.com/2014/08/18/technology/for-big-data-scientists-hurdle-to-insights-is-janitor-work.html)

![data_cleaning_2](data_cleaning_2.png)
[https://www.forbes.com/sites/gilpress/2016/03/23/data-preparation-most-time-consuming-least-enjoyable-data-science-task-survey-says/#7eb223596f63](https://www.forbes.com/sites/gilpress/2016/03/23/data-preparation-most-time-consuming-least-enjoyable-data-science-task-survey-says/#7eb223596f63)


# Question

Do parking spots with EV charging capability perform better than those without?

# Conversion

Definition is:

$$
C = \frac{R}{S}
$$

where $C$ is conversion rate, $R$ is number of rentals, $S$ is number of searches.

# Populations

So, let's explore our populations: 

- Parking spots with EV charging amenities
- Parking spots without EV charging amenities

In [14]:
from awshelpers import rshelp
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [15]:
rshelp.query("SELECT parking_spot_id, ev_charging FROM postgres_public.parking_spot ORDER BY RANDOM() LIMIT 100;")

Query pid: 10555


,parking_spot_id,ev_charging
0,837,None
1,1483,None
2,79,None
3,1037,None
4,3453,None
5,2188,None
6,470,None
7,2689,None
8,3120,None
9,2886,None


In [2]:
rshelp.query("SELECT COUNT(*) FROM postgres_public.parking_spot WHERE ev_charging IS NOT NULL;")

Query pid: 9100


,count
0,0


Oh... well then. How do I find out if a facility has EV charging?

### Ask an engineer!

Turns out, amenities are no longer tied to facilities, they're tied to a particular rate. You could imagine that EV charging would be offered for reservations of 12 hours or less, but they aren't listed for a 24-hour rental, for instance. That being the case, we needed the flexibility to have amenities change by reservation. So, now our data gets more complicated.

In [16]:
rshelp.query("SELECT id, ev_charging FROM postgres_public.ratings_amenities LIMIT 100;")

Query pid: 10599


,id,ev_charging
0,4929,False
1,1140,False
2,4256,False
3,1977,True
4,1543,False
5,7678,True
6,2899,False
7,4223,False
8,6199,False
9,4887,False


In [4]:
rshelp.query("SELECT COUNT(*) FROM (SELECT * FROM postgres_public.ratings_amenities ORDER BY RANDOM() LIMIT 100) WHERE ev_charging;")

Query pid: 9104


,count
0,2


There's the data!

But... What does it mean? Which facility does it apply to?

# Rentals

Let's look at rentals for a bit.

In [17]:
rshelp.query("""
    SELECT rental_id, renter_id, parking_spot_id, rule_trail 
    FROM postgres_public.rental 
    WHERE created >= '2017-04-01' 
    ORDER BY RANDOM() LIMIT 100;
    """)

Query pid: 10744


,rental_id,renter_id,parking_spot_id,rule_trail
0,6142487,1803060,2971,61608.61608.61608.61608.61608.61608.61608.6160...
1,6155457,1752830,5207,53228
2,6086177,1552311,1344,8326
3,6021948,1772185,4604,71223
4,6095317,1790176,3000,62508.62508.62507.62507.62507.62507.62508.6250...
5,6098892,520460,2749,21031
6,6061015,69279,4264,40769
7,6137541,1801097,2920,23364
8,6085739,1776661,2422,17640
9,6117706,1622399,401,68738


That `rule_trail` field is the one I want, believe it or not. The engineers tell me so. That will join to `id` in `ratings_amenities`. 

But... what's the deal with that `64086.64084` entry? Ugh... "trail" means that multiple ratings can be used for an individual reservation. Each could have different amenities, in principle.

You're telling me I have to write a query that will split `rule_trail` on the `.` and then join each of those to `ratings_amenities` and then run a `BOOL_AND` on the `ev_charging` field in `ratings_amenities`???

The answer is yes. This is what you have to do. But, before getting into that, let's see how the rest of the data will play. I now know that it's possible to get a number for the rentals $R$ above, split on whether this rental offered EV charging or not. But, what about the number of searches $S$?

# Searches

The search data is large. Because of its log-like nature, it's the only part of our core data that one could almost call "Big" Data. You shouldn't call it that because it's a garbage term, but you could. 

This being the case, there have been headaches as to how to architect this data, how to run it, what architecture to support it, etc. It's still a work in progress, as you'll see.

In [21]:
# This takes TOO LONG!
# rshelp.query("SELECT * FROM tracking_public.tracking_results WHERE created >= '2017-04-01' ORDER BY RANDOM() LIMIT 100;")

# Partition it by week?
rshelp.query("""
    SELECT search_id, action_id, spot_id, rule_id
    FROM tracking_public.tracking_results_2017_13 
    ORDER BY RANDOM() LIMIT 100;
    """)

Query pid: 11030


,search_id,action_id,spot_id,rule_id
0,52af479c-1314-11e7-890c-0242ac110002,32c52851-a876-451d-a8e6-a5696aede617,2140,14017.0
1,de1c3fa0-1496-11e7-aeb5-0242ac110002,82185968-a4d4-454b-af08-dbaf37265661,5057,50425.0
2,d686b8ea-17f9-11e7-893f-0242ac110002,da9275a7-cc4e-4b29-95be-44b870266473,994,62071.0
3,ae69cf4a-1403-11e7-8d13-0242ac110002,ae9c5359-1c68-4fcd-b40d-01e1b3a5762e,1057,21581.0
4,5e191864-16d5-11e7-8f63-0242ac110002,5e1919ea-16d5-11e7-8f63-0242ac110002,4578,57545.0
5,ebb6aed6-1480-11e7-a2c7-0242ac110002,ebb6b124-1480-11e7-a2c7-0242ac110002,5852,NaN
6,5ec28b05-080e-459e-b486-316850cac0bf,47b517f2-814d-434e-a380-3eef3d5dfb45,1509,14517.0
7,e8c9db1e-1401-11e7-b61b-0242ac110002,93df2dbe-d758-4cea-a19c-29f8543a3b2b,4484,NaN
8,c520756a-1622-11e7-a0ea-0242ac110002,fb3e665e-77be-44b8-86e7-e4016c477cc4,2557,70392.0
9,9586008e-16ab-11e7-8180-0242ac110002,9586020a-16ab-11e7-8180-0242ac110002,1771,14966.0


Well, look at that: `rule_id` is in there! Wait... shouldn't it also be a `rule_trail`? And what is with those `null` values? 

Oh no... that field is defined as an `int` in the database, isn't it? It can't handle a value like `64086.64084`, and so it writes as a `null`, doesn't it? 

### Ask an engineer!

Can we fix this field, make it a `string`?

Sure!

Can we backfill that data and make it correct historically? 

Most definitely not! That ship has sailed.

Okay then... so, let's write an engineering ticket to fix this going forward. They'll get to it in 3 sprints, or 6 weeks. In the mean time, to use the historical data we do have, let's make some data archaeology decisions and decide instead of using amenities per search, let's just assume that every search at a facility instead displays it's "default" amenities. I know this is wrong, but it's an assumption we have to make, or else we are going to get stuck in traffic.

# Default Amenities

Let's look at those parking spots from above. See that `default_amenities_id` in there?

In [22]:
rshelp.query("""
    SELECT parking_spot_id, default_amenities_id 
    FROM postgres_public.parking_spot 
    ORDER BY RANDOM() LIMIT 100;
    """)

Query pid: 11152


,parking_spot_id,default_amenities_id
0,1076,2873.0
1,5893,8755.0
2,5335,8084.0
3,2741,1377.0
4,6306,9219.0
5,5691,8496.0
6,2979,1160.0
7,5538,8302.0
8,3813,421.0
9,6074,8948.0


In [9]:
rshelp.query("""
    SELECT ps.parking_spot_id, ra.ev_charging
    FROM postgres_public.parking_spot ps 
    JOIN postgres_public.ratings_amenities ra ON ra.id = ps.default_amenities_id
    ORDER BY RANDOM() LIMIT 100;
    """)

Query pid: 9267


,parking_spot_id,ev_charging
0,4578,False
1,2368,False
2,519,False
3,312,False
4,4712,False
5,5920,False
6,1738,False
7,521,False
8,1799,False
9,1323,False


Yay! We have a good answer for our population division! Finally!

...

...

No wait... Both the `parking_spot` and `ratings_amenities` tables are current snapshots. They tell me if a `parking_spot_id` is `ev_charging` *today*, but what about historically? 

### Ask an engineer!

Is there a way to find out what these tables looked like in the past?

I mean... yes... but, you're not going to like it.

# Revision Tracking

So, there are these two tables called `reversion_version` and `reversion_revision`. I am not joking. Let's look at what's in there.

In [24]:
rshelp.query("""
    SELECT rr.date_created, rv.object_id_int, rv.serialized_data
    FROM postgres_public.reversion_version rv
    JOIN postgres_public.reversion_revision rr ON rr.id = rv.revision_id
    WHERE rv.content_type_id = 28
    ORDER BY RANDOM() LIMIT 100;
    """)

Query pid: 12366


,date_created,object_id_int,serialized_data
0,2015-11-11 15:57:39.843,1157,"[{""fields"": {""vehicle_size"": ""oversized_with_f..."
1,2016-09-21 18:39:02.028,2704,"[{""fields"": {""vehicle_size"": ""not_applicable"",..."
2,2015-11-04 19:09:54.303,3209,"[{""fields"": {""vehicle_size"": ""not_applicable"",..."
3,2014-10-01 16:52:33.661,1609,"[{""pk"": 1609, ""model"": ""spothero.parkingspot"",..."
4,2016-09-27 21:17:49.265,2785,"[{""fields"": {""vehicle_size"": ""oversized_with_f..."
5,2015-05-13 20:30:30.135,1146,"[{""pk"": 1146, ""model"": ""spothero.parkingspot"",..."
6,2016-09-20 19:42:39.449,487,"[{""fields"": {""vehicle_size"": ""not_applicable"",..."
7,2016-12-05 03:35:11.764,5801,"[{""fields"": {""vehicle_size"": ""not_applicable"",..."
8,2015-08-04 14:29:32.892,1147,"[{""fields"": {""vehicle_size"": ""oversized"", ""pho..."
9,2016-11-09 17:25:46.559,4510,"[{""fields"": {""vehicle_size"": ""not_applicable"",..."


My God... 

😎🕶😦

Is that... JSON? As a field in a relational database?

BAHAHHAHAHAHHA! YOU WISH! If it were nice, valid JSON, there are built in Redshift functions to parse that. 

In [11]:
rshelp.query("""
    SELECT 
        rr.date_created,
        rv.serialized_data,
        JSON_EXTRACT_PATH_TEXT(rv.serialized_data, 'fields', 'default_amenities')
    FROM postgres_public.reversion_version rv
    JOIN postgres_public.reversion_revision rr ON rr.id = rv.revision_id
    WHERE rv.content_type_id = 28
          AND rr.date_created >= '2017-04-01'
    ORDER BY RANDOM() LIMIT 100;
    """)

Query pid: 9382


ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 4))



InternalError: JSON parsing error
DETAIL:  
  -----------------------------------------------
  error:  JSON parsing error
  code:      8001
  context:   invalid json object [{"model": "spothero.parkingspot", "pk": 4625, "fields": {"created": "2016-05-27T19:36:19.578Z", "title": "86 E Wacker Pl. - Comfort Suites Valet Stand", "description": "", "post_purchase_instructions": "", "spaces": 10, "dynamic_invent
  query:     7676528
  location:  funcs_json.h:119
  process:   query3_78 [pid=4591]
  -----------------------------------------------



![malformed_json](malformed_json.png)

The answer... well, it is not fun. It involves writing a Redshift user-defined function (UDF) to parse through this JSON-like blob of text using regex, but in a way that can handle errors gracefully. The code looked like this:

In [12]:
create_udf__get_refund_path = """
    CREATE OR REPLACE FUNCTION warehouse.fuzzy_parse_json(json_blob varchar(65535),
                                                          path_list varchar(65535))
    RETURNS varchar(65535)
    IMMUTABLE
    AS $$
    # Pass json_blob field and a path_list that must be of the form:
    #   '["key1", "key2", 1, ...]' where each entry in the list-like string is either
    #   a string enclosed in "", which is treated as a key for a following dictionary
    #   or
    #   an integer, which is treated as an index for a following list or tuple
        import ast
        import re
        import json

        # Turn the path_list string argument into a literal list to iterate over
        path_list = ast.literal_eval(path_list)

        # Just make sure it is not valid JSON first, because this is easy if so
        try:
            value_out = json.loads(json_blob)
            for p in path_list:
                value_out = value_out[p]
            return value_out
        except:
            pass

        # JSON iterable encloser pairs
        enclosers = {'{': '}', '[': ']', '(': ')'}

        # The value to continually trim. At each step, value_out is what remains of the json_blob.
        # An implied list of a single dict, so do not require always leading path_list with 0.
        value_out = json_blob[1:-1]

        # Get the next key or index from the path_list
        for p in path_list:
            #
            # If value_out is list- or tuple-like, accept an integer in path_list
            #
            if re.search(r'(^\[(.*)\]$)|(^\((.*)\)$)', value_out):
                if not isinstance(p, int):
                    return None
                # Accept index if p is an int and a list- or tuple-like string remains
                i = 0
                buffer = ''
                found = False
                # Walk through each ',' split string segment
                for x in value_out[1:-1].split(','):
                    # Buffer from previous step, if what was split was not a complete object
                    if buffer:
                        # Try to complete the object by adding this step to previous step(s)
                        x = ','.join((buffer, x))
                    for o, c in enclosers.iteritems():
                        # Commas could be enclosed in a nested object, and should be ignored.
                        # This is how we check that every open encloser is met with its closer.
                        if (x.count(o) != x.count(c)) or (x.count('"') % 2 != 0):
                            # If any object was not closed, then the comma that was split on
                            #   should have been nested, so add then next step to this step.
                            buffer = x
                            break
                        buffer = ''
                    if buffer == '':
                        # If it was a full object returned, this is the ith element.
                        # Is the ith element the one you asked for (p)?
                        if p == i:
                            value_out = x.strip()
                            found = True
                            break
                        i += 1
                if (not found) or (value_out == ''):
                    return None
                # If this was our choice to trim value_out, continue on to the next p.
                continue
            #
            # If value_out is dict-like, accept a str- or int-like key
            #
            elif re.search(r'^\{(.*)\}$', value_out):
                if isinstance(p, str):
                    # A str-like key is actually surrounded by ""
                    p = '"{}"'.format(p)
                elif not isinstance(p, int):
                    return None
                # Ensure there is a key match in
                if re.search(r'(?<=(\{{| |,){}:).*'.format(p), value_out) is None:
                    return None
                # Find the key in this dictionary
                found_str = re.search(r'(?<=(\{{| |,){}:).*'.format(p), value_out).group(0).strip()
                # If the value for this key is dict-, list-, or tuple-like, ignore those nested
                # Protect for bad casting of iterables into strings, e.g. "["field"]"
                if (found_str[0] == '"') and (found_str[1] in enclosers.keys()):
                    found_str = found_str[1:]
                if found_str[0] in enclosers.keys():
                    container_count = 1
                    o = found_str[0]
                    c = enclosers[o]
                    value_out = o
                    sub_list = found_str[1:].split(c)
                    while (container_count > 0) and (sub_list not in ('', [''])):
                        x = sub_list.pop(0)
                        container_count -= 1
                        value_out += x + c
                        container_count += x.count(o)
                    if value_out.count(o) != value_out.count(c):
                        return None
                # If the value for this key is string-like, get this string
                elif found_str[0] == '"':
                    value_out = found_str[1:].split('"')[0]
                # Otherwise, the value is object-like (bool, int, float, null, etc), get this obj
                else:
                    value_out = found_str.split(',')[0]
                    if value_out == 'null':
                        value_out = None
            else:
                return None
        return value_out
    $$ LANGUAGE plpythonu;
"""

In [25]:
rshelp.query("""
    SELECT 
        rr.date_created AS date_of_change,
        rv.object_id_int AS parking_spot_id,
        warehouse.fuzzy_parse_json(rv.serialized_data, '["fields", "default_amenities"]') AS default_amenities_id
    FROM postgres_public.reversion_version rv
    JOIN postgres_public.reversion_revision rr ON rr.id = rv.revision_id
    WHERE rv.content_type_id = 28
          AND rr.date_created >= '2017-04-01'
    ORDER BY RANDOM() LIMIT 100;
    """)

Query pid: 12521


,date_of_change,parking_spot_id,default_amenities_id
0,2017-04-02 17:32:37.583,1822,2190
1,2017-04-08 19:37:17.931,4833,7525
2,2017-04-06 14:13:57.107,6376,9309
3,2017-04-01 14:03:10.415,5228,7967
4,2017-04-09 20:52:21.664,2576,4796
5,2017-04-06 18:41:46.625,6360,9289
6,2017-04-13 16:31:17.761,1435,7110
7,2017-04-09 13:00:33.450,3641,7231
8,2017-04-12 14:23:46.677,6425,9368
9,2017-04-07 20:57:19.086,1633,2363


What you see above is what pure, utter joy in Data Science looks like. I am sorry, but this is true. It is amazing that I was just able to do that. Literally just writing this one function and deploying it to the analyst team will save time and energy and provide tons of value to each of the business units.

# So, I'm going to stop here...

Here's where we are:

We have the ability to find the number of rentals $R$ by `parking_spot_id` and the ability to tell whether `ev_charging` was enabled at the time of purchase. We have the same for number of searches $S$. 

What we still haven't considered:

- Can I join each rental to a search to determine if this particular search converted? 
    - NO! We're missing that data on the `rental` data sometimes.
- Is there something fishy with the default behavior in searches that would make we want to subset which searches I'm even considering?
    - YES! On our web product, the initial search defaults to current time, 3-hr reservation. When a user changes their time preference, it creates a new `search_id` in our backend. Thus, those 3-hr searches dominate our data but are mostly abandoned, by definition. But, only on web, mind you.
- Is the location distribution of facilities with EV charging enabled similar to a random sample of locations? 
    - NO! They dominate places like the Loop, so we'd really want to find populations of facilities with EV charging stations that have a similar distance distribution to where users are searching to another test population. Calculating where a user is trying to go and then getting the distance between that target location and the location of the facility is... not trivial.
- Is the price distribution of facilities with EV charging enabled similar to a random sample of locations? 
    - NO! Same deal as above.
- Do price and distance dominate our conversion rate expectations? 
    - OF COURSE! To see the signal of "Does EV charging impact conversion rates?" above the variation in conversion rates caused by other factors is... not trivial.

And on and on it goes...